In [1]:
from __future__ import division

import keras
import h5py
from PIL import Image
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D
#from keras.utils.visualize_util import plot
from keras.utils import np_utils

from keras.utils.vis_utils import model_to_dot, plot_model
from IPython.display import SVG

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
#keras.backend.image_dim_ordering='th'


from keras import backend as K
import theano
theano.config.optimizer='fast_compile'

Using TensorFlow backend.


In [2]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense

vgg16 = VGG16(weights='imagenet')

In [3]:
for layer in vgg16.layers[:15]:
    layer.trainable = False
    
convs = vgg16.get_layer('flatten').output

#create new layers
new = Dense(1024, activation='relu', name='fc6_pose')(convs)
new = Dense(1024, activation='relu', name='fc7_pose')(new)

# predict locations

# output : 2 visibility probabilities
vis1 = Dense(2, activation='softmax', name='visibility1')(new)
vis2 = Dense(2, activation='softmax', name='visibility2')(new)
vis3 = Dense(2, activation='softmax', name='visibility3')(new)
vis4 = Dense(2, activation='softmax', name='visibility4')(new)
vis5 = Dense(2, activation='softmax', name='visibility5')(new)
vis6 = Dense(2, activation='softmax', name='visibility6')(new)

# landmark outputs
lm=Dense(12,activation='linear',name='landmarks')(new)

outputs=[vis1, vis2, vis3, vis4,vis5, vis6, lm]

model = Model(input=vgg16.input, output=outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

C:\Users\1023473\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`


In [24]:
from keras import optimizers
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam, loss={'visibility1':'binary_crossentropy','visibility2':'binary_crossentropy','visibility3':'binary_crossentropy',
        'visibility4':'binary_crossentropy','visibility5':'binary_crossentropy',
'visibility6':'binary_crossentropy', 'landmarks': 'mean_squared_error'}, metrics=['accuracy'])

In [5]:
import pandas as pd

lm='C:/Users/1023473/Downloads/Anno/list_landmarks.txt'
text=pd.read_csv(lm,delim_whitespace=True,skiprows=0,header=1)


text.tail()

,image_name,clothes_type,variation_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,landmark_visibility_2,landmark_location_x_2,landmark_location_y_2,landmark_visibility_3,...,landmark_location_y_5,landmark_visibility_6,landmark_location_x_6,landmark_location_y_6,landmark_visibility_7,landmark_location_x_7,landmark_location_y_7,landmark_visibility_8,landmark_location_x_8,landmark_location_y_8
123011,img/img_00123055.jpg,3,2,1,155,81,0,180,82,0,...,175.0,0.0,193.0,176.0,0.0,115.0,328.0,0.0,214.0,347.0
123012,img/img_00123056.jpg,3,2,1,241,73,1,273,74,0,...,137.0,0.0,296.0,136.0,0.0,211.0,209.0,1.0,290.0,207.0
123013,img/img_00123057.jpg,3,3,1,53,72,1,64,72,0,...,151.0,1.0,72.0,149.0,0.0,18.0,205.0,1.0,85.0,205.0
123014,img/img_00123058.jpg,3,3,1,142,106,0,155,103,1,...,228.0,0.0,151.0,227.0,1.0,130.0,358.0,0.0,183.0,365.0
123015,img/img_00123059.jpg,3,2,0,131,153,1,214,151,0,...,262.0,1.0,214.0,261.0,1.0,103.0,389.0,0.0,227.0,385.0


In [6]:
text.describe()

,clothes_type,variation_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,landmark_visibility_2,landmark_location_x_2,landmark_location_y_2,landmark_visibility_3,landmark_location_x_3,...,landmark_location_y_5,landmark_visibility_6,landmark_location_x_6,landmark_location_y_6,landmark_visibility_7,landmark_location_x_7,landmark_location_y_7,landmark_visibility_8,landmark_location_x_8,landmark_location_y_8
count,123016.000000,123016.000000,123016.000000,123016.000000,123016.000000,123016.000000,123016.000000,123016.000000,123016.000000,123016.000000,...,92044.000000,92044.000000,92044.000000,92044.000000,50013.000000,50013.000000,50013.000000,50013.000000,50013.000000,50013.000000
mean,2.064886,2.449982,0.540434,134.936309,102.988693,0.527777,192.464240,102.264738,0.381170,110.171043,...,245.812611,0.382730,193.914074,245.386337,0.493632,102.095655,308.880311,0.473457,179.657669,308.333293
std,0.862568,1.399137,0.561416,61.939054,70.166523,0.564925,72.965077,70.158381,0.596219,70.089207,...,115.651265,0.619806,84.829415,116.286360,0.756338,73.543060,159.518588,0.759751,102.302270,160.539966
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,98.000000,50.000000,0.000000,151.000000,50.000000,0.000000,65.000000,...,180.000000,0.000000,147.000000,180.000000,0.000000,52.000000,240.000000,0.000000,125.000000,238.000000
50%,2.000000,2.000000,1.000000,133.000000,95.000000,0.000000,195.000000,94.000000,0.000000,105.000000,...,242.000000,0.000000,203.000000,242.000000,0.000000,100.000000,345.000000,0.000000,200.000000,345.000000
75%,3.000000,4.000000,1.000000,165.000000,142.000000,1.000000,232.000000,142.000000,1.000000,145.000000,...,319.000000,1.000000,246.000000,319.000000,1.000000,144.000000,441.000000,1.000000,250.000000,441.000000
max,3.000000,5.000000,2.000000,491.000000,503.000000,2.000000,507.000000,467.000000,2.000000,506.000000,...,512.000000,2.000000,510.000000,512.000000,2.000000,504.000000,512.000000,2.000000,507.000000,512.000000


In [7]:
upper=text[text.clothes_type ==1]

upper.describe()

,clothes_type,variation_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,landmark_visibility_2,landmark_location_x_2,landmark_location_y_2,landmark_visibility_3,landmark_location_x_3,...,landmark_location_y_5,landmark_visibility_6,landmark_location_x_6,landmark_location_y_6,landmark_visibility_7,landmark_location_x_7,landmark_location_y_7,landmark_visibility_8,landmark_location_x_8,landmark_location_y_8
count,42031.0,42031.000000,42031.000000,42031.000000,42031.000000,42031.000000,42031.000000,42031.000000,42031.000000,42031.000000,...,42031.000000,42031.000000,42031.000000,42031.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.0,2.454165,0.388071,139.538222,103.887631,0.361091,191.626466,102.954557,0.360496,98.158336,...,270.961148,0.478837,175.434298,270.841450,NaN,NaN,NaN,NaN,NaN,NaN
std,0.0,1.355310,0.519960,57.437633,56.747877,0.514013,67.190153,56.784962,0.599424,69.565090,...,148.336300,0.753309,101.169926,149.261111,NaN,NaN,NaN,NaN,NaN,NaN
min,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.0,1.000000,0.000000,103.000000,65.000000,0.000000,149.000000,64.000000,0.000000,52.000000,...,191.000000,0.000000,121.000000,190.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.0,2.000000,0.000000,136.000000,102.000000,0.000000,193.000000,101.000000,0.000000,89.000000,...,294.000000,0.000000,190.000000,294.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.0,4.000000,1.000000,166.000000,143.000000,1.000000,227.000000,142.000000,1.000000,131.000000,...,392.000000,1.000000,244.000000,393.000000,NaN,NaN,NaN,NaN,NaN,NaN
max,1.0,5.000000,2.000000,467.000000,461.000000,2.000000,502.000000,451.000000,2.000000,502.000000,...,512.000000,2.000000,510.000000,512.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
upper2=upper[(upper.landmark_visibility_1!=2)&(upper.landmark_visibility_2!=2)&(upper.landmark_visibility_3!=2)&
            (upper.landmark_visibility_4!=2) & (upper.landmark_visibility_5!=2) & (upper.landmark_visibility_6!=2)]

In [9]:
file1='C:/Users/1023473/Downloads/Eval/list_eval_partition.txt'

split=pd.read_csv(file1,delim_whitespace=True,skiprows=[0])

split.head()
len(split)

123016

In [10]:
splits=upper2.merge(split,how='left')

splits.head(1)

len(splits)
def image(x):
    x=x.split('/')[1:]
    return '/'.join(x)

splits['image']=splits.image_name.apply(image)

def filename(df):
    x=df.evaluation_status+'/'+df.image
    return x

splits['file']=splits.apply(filename,axis=1)
splits.head()

,image_name,clothes_type,variation_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,landmark_visibility_2,landmark_location_x_2,landmark_location_y_2,landmark_visibility_3,...,landmark_location_y_6,landmark_visibility_7,landmark_location_x_7,landmark_location_y_7,landmark_visibility_8,landmark_location_x_8,landmark_location_y_8,evaluation_status,image,file
0,img/img_00000001.jpg,1,1,1,144,156,1,221,159,0,...,412.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000001.jpg,train/img_00000001.jpg
1,img/img_00000002.jpg,1,3,0,111,104,1,153,105,0,...,198.0,NaN,NaN,NaN,NaN,NaN,NaN,val,img_00000002.jpg,val/img_00000002.jpg
2,img/img_00000003.jpg,1,1,0,112,113,1,172,117,0,...,262.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000003.jpg,train/img_00000003.jpg
3,img/img_00000004.jpg,1,2,0,76,74,0,108,69,0,...,170.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000004.jpg,train/img_00000004.jpg
4,img/img_00000005.jpg,1,1,0,217,139,0,290,156,0,...,313.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000005.jpg,train/img_00000005.jpg


In [11]:
trainData = splits[splits.	evaluation_status =='train']
valData = splits[splits.	evaluation_status =='val']
trainData.head()

,image_name,clothes_type,variation_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,landmark_visibility_2,landmark_location_x_2,landmark_location_y_2,landmark_visibility_3,...,landmark_location_y_6,landmark_visibility_7,landmark_location_x_7,landmark_location_y_7,landmark_visibility_8,landmark_location_x_8,landmark_location_y_8,evaluation_status,image,file
0,img/img_00000001.jpg,1,1,1,144,156,1,221,159,0,...,412.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000001.jpg,train/img_00000001.jpg
2,img/img_00000003.jpg,1,1,0,112,113,1,172,117,0,...,262.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000003.jpg,train/img_00000003.jpg
3,img/img_00000004.jpg,1,2,0,76,74,0,108,69,0,...,170.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000004.jpg,train/img_00000004.jpg
4,img/img_00000005.jpg,1,1,0,217,139,0,290,156,0,...,313.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000005.jpg,train/img_00000005.jpg
5,img/img_00000006.jpg,1,1,1,63,50,1,98,51,0,...,135.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000006.jpg,train/img_00000006.jpg


In [12]:
splits.head()

,image_name,clothes_type,variation_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,landmark_visibility_2,landmark_location_x_2,landmark_location_y_2,landmark_visibility_3,...,landmark_location_y_6,landmark_visibility_7,landmark_location_x_7,landmark_location_y_7,landmark_visibility_8,landmark_location_x_8,landmark_location_y_8,evaluation_status,image,file
0,img/img_00000001.jpg,1,1,1,144,156,1,221,159,0,...,412.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000001.jpg,train/img_00000001.jpg
1,img/img_00000002.jpg,1,3,0,111,104,1,153,105,0,...,198.0,NaN,NaN,NaN,NaN,NaN,NaN,val,img_00000002.jpg,val/img_00000002.jpg
2,img/img_00000003.jpg,1,1,0,112,113,1,172,117,0,...,262.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000003.jpg,train/img_00000003.jpg
3,img/img_00000004.jpg,1,2,0,76,74,0,108,69,0,...,170.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000004.jpg,train/img_00000004.jpg
4,img/img_00000005.jpg,1,1,0,217,139,0,290,156,0,...,313.0,NaN,NaN,NaN,NaN,NaN,NaN,train,img_00000005.jpg,train/img_00000005.jpg


In [13]:
valData.describe()

,clothes_type,variation_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,landmark_visibility_2,landmark_location_x_2,landmark_location_y_2,landmark_visibility_3,landmark_location_x_3,...,landmark_location_y_5,landmark_visibility_6,landmark_location_x_6,landmark_location_y_6,landmark_visibility_7,landmark_location_x_7,landmark_location_y_7,landmark_visibility_8,landmark_location_x_8,landmark_location_y_8
count,5235.0,5235.000000,5235.000000,5235.000000,5235.00000,5235.000000,5235.000000,5235.000000,5235.000000,5235.000000,...,5235.000000,5235.000000,5235.000000,5235.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.0,1.893983,0.417765,140.669532,106.06915,0.400000,187.496275,105.198090,0.277746,107.320917,...,315.568290,0.201910,202.826934,316.172684,NaN,NaN,NaN,NaN,NaN,NaN
std,0.0,1.038259,0.493238,50.616086,48.91022,0.489945,57.234577,49.017573,0.447930,62.933229,...,99.677415,0.401464,68.778322,99.882874,NaN,NaN,NaN,NaN,NaN,NaN
min,1.0,1.000000,0.000000,16.000000,2.00000,0.000000,29.000000,2.000000,0.000000,3.000000,...,30.000000,0.000000,6.000000,33.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.0,1.000000,0.000000,107.000000,72.00000,0.000000,147.000000,71.000000,0.000000,65.000000,...,229.000000,0.000000,149.000000,230.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.0,2.000000,0.000000,136.000000,106.00000,0.000000,188.000000,105.000000,0.000000,97.000000,...,321.000000,0.000000,202.000000,321.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.0,2.000000,1.000000,164.000000,142.00000,1.000000,219.000000,141.000000,1.000000,136.000000,...,400.000000,0.000000,249.000000,400.000000,NaN,NaN,NaN,NaN,NaN,NaN
max,1.0,5.000000,1.000000,419.000000,316.00000,1.000000,478.000000,317.000000,1.000000,478.000000,...,510.000000,1.000000,499.000000,512.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
valData.head()

,image_name,clothes_type,variation_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,landmark_visibility_2,landmark_location_x_2,landmark_location_y_2,landmark_visibility_3,...,landmark_location_y_6,landmark_visibility_7,landmark_location_x_7,landmark_location_y_7,landmark_visibility_8,landmark_location_x_8,landmark_location_y_8,evaluation_status,image,file
1,img/img_00000002.jpg,1,3,0,111,104,1,153,105,0,...,198.0,NaN,NaN,NaN,NaN,NaN,NaN,val,img_00000002.jpg,val/img_00000002.jpg
11,img/img_00000012.jpg,1,1,1,73,74,1,107,68,0,...,212.0,NaN,NaN,NaN,NaN,NaN,NaN,val,img_00000012.jpg,val/img_00000012.jpg
20,img/img_00000021.jpg,1,1,1,137,156,1,201,159,0,...,338.0,NaN,NaN,NaN,NaN,NaN,NaN,val,img_00000021.jpg,val/img_00000021.jpg
37,img/img_00000060.jpg,1,3,1,138,130,0,152,113,1,...,434.0,NaN,NaN,NaN,NaN,NaN,NaN,val,img_00000060.jpg,val/img_00000060.jpg
40,img/img_00000063.jpg,1,3,1,172,74,1,234,101,0,...,197.0,NaN,NaN,NaN,NaN,NaN,NaN,val,img_00000063.jpg,val/img_00000063.jpg


In [15]:
trainData.describe()

,clothes_type,variation_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,landmark_visibility_2,landmark_location_x_2,landmark_location_y_2,landmark_visibility_3,landmark_location_x_3,...,landmark_location_y_5,landmark_visibility_6,landmark_location_x_6,landmark_location_y_6,landmark_visibility_7,landmark_location_x_7,landmark_location_y_7,landmark_visibility_8,landmark_location_x_8,landmark_location_y_8
count,21488.0,21488.000000,21488.000000,21488.000000,21488.000000,21488.000000,21488.000000,21488.000000,21488.000000,21488.000000,...,21488.000000,21488.000000,21488.000000,21488.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.0,1.900363,0.428984,142.226126,106.865087,0.389939,189.622022,105.885843,0.282390,108.470867,...,318.138263,0.201554,205.037742,319.187500,NaN,NaN,NaN,NaN,NaN,NaN
std,0.0,1.046276,0.494942,51.732500,48.532992,0.487747,58.012483,48.580628,0.450173,63.807687,...,97.492863,0.401170,69.380236,97.777319,NaN,NaN,NaN,NaN,NaN,NaN
min,1.0,1.000000,0.000000,10.000000,1.000000,0.000000,15.000000,1.000000,0.000000,1.000000,...,10.000000,0.000000,5.000000,9.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.0,1.000000,0.000000,108.000000,73.000000,0.000000,150.000000,72.000000,0.000000,65.000000,...,235.000000,0.000000,153.000000,236.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.0,2.000000,0.000000,138.000000,107.000000,0.000000,190.000000,106.000000,0.000000,98.000000,...,325.000000,0.000000,205.000000,327.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.0,2.000000,1.000000,166.000000,143.000000,1.000000,221.000000,142.000000,1.000000,136.000000,...,399.000000,0.000000,248.000000,401.000000,NaN,NaN,NaN,NaN,NaN,NaN
max,1.0,5.000000,1.000000,467.000000,344.000000,1.000000,469.000000,376.000000,1.000000,500.000000,...,512.000000,1.000000,499.000000,509.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
train_num = len(trainData.index)
val_num = len(valData.index)
#hf = h5py.File('data.h5', 'w')
v1 = np.zeros((train_num,2))
v2 = np.zeros((train_num,2))
v3 = np.zeros((train_num,2))
v4 = np.zeros((train_num,2))
v5 = np.zeros((train_num,2))
v6 = np.zeros((train_num,2))
lm = np.zeros((train_num,12))

In [17]:
i=0
train_num = len(trainData.index)
lm = np.zeros((train_num,12))
for index, row in trainData.iterrows():
    im = Image.open(row['file'])
    width, height = im.size
    lm[i][0] = (row["landmark_location_x_1"]/width)
    lm[i][2] = (row["landmark_location_x_2"]/width)
    lm[i][4] = (row["landmark_location_x_3"]/width)
    lm[i][6] = (row["landmark_location_x_4"]/width)
    lm[i][8] = (row["landmark_location_x_5"]/width)
    lm[i][10] = (row["landmark_location_x_6"]/width)
    lm[i][1] = (row["landmark_location_y_1"]/height)
    lm[i][3] = (row["landmark_location_y_2"]/height)
    lm[i][5] = (row["landmark_location_y_3"]/height)
    lm[i][7] = (row["landmark_location_y_4"]/height)
    lm[i][9] = (row["landmark_location_y_5"]/height)
    lm[i][11] = (row["landmark_location_y_6"]/height)
    i+=1

In [18]:
lm

array([[0.41260745, 0.3046875 , 0.63323782, ..., 0.796875  , 0.67908309,
        0.8046875 ],
       [0.36363636, 0.25      , 0.55844156, ..., 0.61283186, 0.60714286,
        0.57964602],
       [0.3877551 , 0.296     , 0.55102041, ..., 0.696     , 0.64285714,
        0.68      ],
       ...,
       [0.41927083, 0.2890625 , 0.59895833, ..., 0.8359375 , 0.57552083,
        0.88476562],
       [0.535     , 0.228     , 0.635     , ..., 0.628     , 0.71      ,
        0.62      ],
       [0.46666667, 0.3       , 0.62      , ..., 0.5575    , 0.62333333,
        0.5775    ]])

In [19]:
i=0
for index, row in trainData.iterrows():
    if row["landmark_visibility_1"] == 0 :
        v1[i][1] = 1
    else:
        v1[i][0] = 1
    i+=1
i=0
for index, row in trainData.iterrows():
    if row["landmark_visibility_2"] == 0 :
        v2[i][1] = 1
    else:
        v2[i][0] = 1
    i+=1
i=0
for index, row in trainData.iterrows():
    if row["landmark_visibility_3"] == 0 :
        v3[i][1] = 1
    else:
        v3[i][0] = 1
    i+=1
i=0
for index, row in trainData.iterrows():
    if row["landmark_visibility_4"] == 0 :
        v4[i][1] = 1
    else:
        v4[i][0] = 1
    i+=1
i=0
for index, row in trainData.iterrows():
    if row["landmark_visibility_5"] == 0 :
        v5[i][1] = 1
    else:
        v5[i][0] = 1
    i+=1
i=0
for index, row in trainData.iterrows():
    if row["landmark_visibility_6"] == 0 :
        v6[i][1] = 1
    else:
        v6[i][0] = 1
    i+=1


In [20]:
def preprocess_input(path):
    img = keras.preprocessing.image.load_img(path, target_size=(224, 224))
    x = keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)    
    return x

In [33]:
def multi_output_generator(data,v1,v2,v3,v4,v5,v6,lms, nb_data, batch_size):
    """ Generates batches of tensor image data in form of ==> x, [y1, y2, y3, y4, y5] for use in a multi-output Keras model.

        # Arguments
            hdf5_file: the hdf5 file which contains the images and the annotations.
            nb_data: total number of samples saved in the array.
            batch_size: size of the batch to generate tensor image data for.

        # Returns
            A five-output generator.
    """

    batches_list = list(range(int(math.ceil(float(nb_data) / batch_size))))

    while True:

        # loop over batches
        for n, i in enumerate(batches_list):
            x_batch = np.zeros(shape=(batch_size, 224, 224, 3))
            i_s = i * batch_size  # index of the first image in this batch
            i_e = min([(i + 1) * batch_size, nb_data])  # index of the last image in this batch
            if ((i+1)*batch_size) > (nb_data-1):
                break
            #x = hdf5_file["file"][i_s:i_e, ...]
            for j in range(batch_size):
                img_name = data.iloc[i * batch_size + j]['file']
                x = preprocess_input(img_name)
                x_batch[j, :, :, :] = x
            # read labels
#            y1 = hdf5_file["landmark_visibility_1"][i_s:i_e]
#            y2 = hdf5_file["landmark_visibility_2"][i_s:i_e]
#            y3 = hdf5_file["landmark_visibility_3"][i_s:i_e]
#            y4 = hdf5_file["landmark_visibility_4"][i_s:i_e]
#            y5 = hdf5_file["landmark_visibility_5"][i_s:i_e]
#            y6 = hdf5_file["landmark_visibility_6"][i_s:i_e]        
            y1 = v1[i_s:i_e]
            y2 = v2[i_s:i_e]
            y3 = v3[i_s:i_e]
            y4 = v4[i_s:i_e]
            y5 = v5[i_s:i_e]
            y6 = v6[i_s:i_e]
            lmr = lms[i_s:i_e]
            
        yield( x_batch, {'visibility1' : y1, 'visibility2' : y2, 'visibility3': y3,'visibility4': y4 ,'visibility5':y5, 'visibility6':y6, 'landmarks':lmr})

In [22]:
from keras.models import Model
from keras.layers import Input, Dense
import math

In [26]:
history = model.fit_generator(multi_output_generator(trainData,v1,v2,v3,v4,v5,v6,lm, 64, 32), steps_per_epoch=2, epochs=10, verbose=2, callbacks=None, validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=False, initial_epoch=0)


Epoch 1/10
 - 64s - loss: 6.6789 - visibility1_loss: 0.6845 - visibility2_loss: 0.7504 - visibility3_loss: 0.6882 - visibility4_loss: 0.6216 - visibility5_loss: 0.8968 - visibility6_loss: 0.4342 - landmarks_loss: 2.6033 - visibility1_acc: 0.6250 - visibility2_acc: 0.5312 - visibility3_acc: 0.5625 - visibility4_acc: 0.6875 - visibility5_acc: 0.7812 - visibility6_acc: 0.8438 - landmarks_acc: 0.1406
Epoch 2/10
 - 61s - loss: 4.2194 - visibility1_loss: 0.7036 - visibility2_loss: 0.7373 - visibility3_loss: 0.7420 - visibility4_loss: 0.7125 - visibility5_loss: 0.5601 - visibility6_loss: 0.5596 - landmarks_loss: 0.2043 - visibility1_acc: 0.3750 - visibility2_acc: 0.5000 - visibility3_acc: 0.4375 - visibility4_acc: 0.3125 - visibility5_acc: 0.7812 - visibility6_acc: 0.8438 - landmarks_acc: 0.1719
Epoch 3/10
 - 61s - loss: 8.6791 - visibility1_loss: 1.4029 - visibility2_loss: 0.9038 - visibility3_loss: 1.2290 - visibility4_loss: 1.4603 - visibility5_loss: 0.8470 - visibility6_loss: 0.8816 - lan

In [27]:
history = model.fit_generator(multi_output_generator(trainData,v1,v2,v3,v4,v5,v6,lm, 64, 32), steps_per_epoch=2, epochs=10, verbose=2, callbacks=None, validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=False, initial_epoch=0)


Epoch 1/10
 - 70s - loss: 3.8347 - visibility1_loss: 0.6785 - visibility2_loss: 0.6913 - visibility3_loss: 0.6903 - visibility4_loss: 0.6416 - visibility5_loss: 0.5664 - visibility6_loss: 0.4902 - landmarks_loss: 0.0764 - visibility1_acc: 0.6250 - visibility2_acc: 0.5312 - visibility3_acc: 0.5625 - visibility4_acc: 0.6875 - visibility5_acc: 0.7812 - visibility6_acc: 0.8438 - landmarks_acc: 0.0000e+00
Epoch 2/10
 - 59s - loss: 3.7643 - visibility1_loss: 0.6639 - visibility2_loss: 0.6936 - visibility3_loss: 0.6860 - visibility4_loss: 0.6245 - visibility5_loss: 0.5406 - visibility6_loss: 0.4730 - landmarks_loss: 0.0826 - visibility1_acc: 0.6250 - visibility2_acc: 0.5312 - visibility3_acc: 0.5625 - visibility4_acc: 0.6875 - visibility5_acc: 0.7812 - visibility6_acc: 0.8438 - landmarks_acc: 0.2812
Epoch 3/10
 - 58s - loss: 3.6931 - visibility1_loss: 0.6641 - visibility2_loss: 0.6919 - visibility3_loss: 0.6860 - visibility4_loss: 0.6265 - visibility5_loss: 0.5432 - visibility6_loss: 0.4462 -

In [34]:
history = model.fit_generator(multi_output_generator(trainData,v1,v2,v3,v4,v5,v6,lm, 400, 25), steps_per_epoch=16, epochs=10, verbose=2, callbacks=None, validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=False, initial_epoch=0)


Epoch 1/10
 - 375s - loss: 3.5604 - visibility1_loss: 0.6682 - visibility2_loss: 0.6779 - visibility3_loss: 0.6315 - visibility4_loss: 0.5544 - visibility5_loss: 0.5032 - visibility6_loss: 0.5039 - landmarks_loss: 0.0213 - visibility1_acc: 0.6225 - visibility2_acc: 0.5875 - visibility3_acc: 0.6800 - visibility4_acc: 0.7600 - visibility5_acc: 0.8000 - visibility6_acc: 0.8000 - landmarks_acc: 0.2800
Epoch 2/10
 - 367s - loss: 3.5227 - visibility1_loss: 0.6536 - visibility2_loss: 0.6734 - visibility3_loss: 0.6271 - visibility4_loss: 0.5515 - visibility5_loss: 0.5011 - visibility6_loss: 0.5011 - landmarks_loss: 0.0149 - visibility1_acc: 0.6400 - visibility2_acc: 0.6000 - visibility3_acc: 0.6800 - visibility4_acc: 0.7600 - visibility5_acc: 0.8000 - visibility6_acc: 0.8000 - landmarks_acc: 0.3325
Epoch 3/10
 - 376s - loss: 3.5195 - visibility1_loss: 0.6535 - visibility2_loss: 0.6730 - visibility3_loss: 0.6269 - visibility4_loss: 0.5512 - visibility5_loss: 0.5005 - visibility6_loss: 0.5005 - 